In [2]:
import MySQLdb
import sys
import bz2
import pandas as pd

In [3]:
# Virtual machine IP adress
HOST            = "148.60.11.195"

## MySQL user
DB_USER         = "script2"

## MySQL password
DB_PASSWD       = "ud6cw3xNRKnrOz6H"

In [ ]:
#-------------------- Get sets of identical configs and save result Dataframe ---------------------
try:
    socket = MySQLdb.connect(HOST, DB_USER, DB_PASSWD, "IrmaDB_result")
    cursor = socket.cursor()
    # firstly select the "basis" config files
    query="SELECT cid, config_file, compiled_kernel_size, compilation_time, hid FROM `compilations` where compiled_kernel_version='4.15' and (cid+200)%400=0 and cid<72000"
    cursor.execute(query)
    result= cursor.fetchall()
    result=[list(i) for i in result]
    df = pd.DataFrame()
    # For each of the previously found cid (i.e our sample), we need to find the entries 
    # which have used the same .config file
    for row in result:
        query2="SELECT cid, compiled_kernel_size, compilation_time, hid FROM compilations \
                WHERE config_file = \
                    (SELECT config_file FROM compilations WHERE cid={})\
                    and cid!={}".format(row[0], row[0])
        cursor.execute(query2)
        matches=cursor.fetchall()
        # dumb conversion tuple -> list for format issues
        matches=[list(i) for i in matches]
        for i in matches:
            i.insert(0, row[0])
        labels=[i[0] for i in cursor.description]
        labels.insert(0, 'origin_config')
        temp=pd.DataFrame(matches, columns=labels)
        df=df.append(temp)
    df.to_pickle('results_latest_2')
except Exception as e:
    print(str(e),"\n" + "Unable to connect to database", file=sys.stderr)
    exit(-1)
finally:
    cursor.close()
    socket.close()
df